In [ ]:
## setup analysis
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.purification import purify_pq as ppq; reload(ppq)
from analysis.lib.purification import purify_analysis as pa; reload(pa)
%matplotlib inline

folder_lt3 = r'D:\measuring\data\Purification_lt3_raw_data'
folder_lt4 = r'D:\measuring\data\Purification_lt4_raw_data'
# addendum = r'\Purify_XX_First_Attempts'
# addendum = r'\Purify_25percent_theta'
#addendum = r'\Purify_35percent_theta'
addendum = r'\Purify_15percent_theta'
folder_lt3 = folder_lt3+addendum
folder_lt4 = folder_lt4+addendum
### ssro calibrations
day = '20160716_' # for ssro calibration

ssro_calib_lt3 = day+'091002'
ssro_calib_lt4 = day+'113952'

Purify_XX = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
Purify_YY = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
Purify_ZZ = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)

In [ ]:
### find timestamps for all three experiments (for pi/6)
# ZZ_days = ['20160808','20160807','20160722','20160721','20160720']
# XX_days = ZZ_days
# YY_days = ['20160808','20160807','20160722','20160724','20160725']
# ZZ_days_new_timings, XX_days_new_timings, YY_days_new_timings = ['20160811'],['20160811'],['20160811']

### for the pi/4 data
# ZZ_days = ['20160714','20160715','20160716','20160718','20160719']
# XX_days = ZZ_days
# YY_days = ['20160727','20160726']
# ZZ_days_new_timings, XX_days_new_timings, YY_days_new_timings = ['20160816'],['20160816'],['20160816']


### for pi/5
# ZZ_days_new_timings = ['20160818','20160819']; YY_days_new_timings = ZZ_days_new_timings; XX_days_new_timings = ZZ_days_new_timings
# ZZ_days = []; YY_days = []; XX_days = []

### for pi/8 
ZZ_days_new_timings = ['20160820','20160821','20160822']; YY_days_new_timings = ZZ_days_new_timings; XX_days_new_timings = ZZ_days_new_timings
ZZ_days = []; YY_days = []; XX_days = []

all_lt3_ZZ , all_lt4_ZZ = [],[]
all_lt3_XX , all_lt4_XX = [],[]
all_lt3_YY , all_lt4_YY = [],[]

offsets_XX,offsets_YY,offsets_ZZ = [],[],[] # Hold offset to compensate for timing change for new APD
correction_time = 2772.5e3-2791.5e3
offsets_ch1_XX,offsets_ch1_YY,offsets_ch1_ZZ = [],[],[]
start_offset_ch1 = 1.8e3

for d in ZZ_days:

    tstamp_lt3,tstamp_lt4 = Purify_ZZ.tstamps_for_both_setups(d,contains = 'ZZ')
    #,newest_tstamp = '110000') ### newest timestamp allows for only taking parts of a day.
    all_lt3_ZZ.extend(tstamp_lt3)
    all_lt4_ZZ.extend(tstamp_lt4)
    offsets_ZZ.extend(np.zeros(np.shape(tstamp_lt3)))
    offsets_ch1_ZZ.extend(np.zeros(np.shape(tstamp_lt4)))
    
for d in YY_days:

    tstamp_lt3,tstamp_lt4 = Purify_YY.tstamps_for_both_setups(d,contains = 'YY')
    all_lt3_YY.extend(tstamp_lt3)
    all_lt4_YY.extend(tstamp_lt4)
    offsets_YY.extend(np.zeros(np.shape(tstamp_lt3)))
    offsets_ch1_YY.extend(np.zeros(np.shape(tstamp_lt4)))
    
for d in XX_days:

    tstamp_lt3,tstamp_lt4 = Purify_XX.tstamps_for_both_setups(d,contains = 'XX')
    all_lt3_XX.extend(tstamp_lt3)
    all_lt4_XX.extend(tstamp_lt4)
    offsets_XX.extend(np.zeros(np.shape(tstamp_lt3)))
    offsets_ch1_XX.extend(np.zeros(np.shape(tstamp_lt4)))
    
for d in ZZ_days_new_timings:

    tstamp_lt3,tstamp_lt4 = Purify_ZZ.tstamps_for_both_setups(d,contains = 'ZZ')
    #,newest_tstamp = '110000') ### newest timestamp allows for only taking parts of a day.
    all_lt3_ZZ.extend(tstamp_lt3)
    all_lt4_ZZ.extend(tstamp_lt4)
    offsets_ZZ.extend(np.zeros(np.shape(tstamp_lt3))+ correction_time)
    offsets_ch1_ZZ.extend(np.zeros(np.shape(tstamp_lt4))+ start_offset_ch1)
    
for d in YY_days_new_timings:

    tstamp_lt3,tstamp_lt4 = Purify_YY.tstamps_for_both_setups(d,contains = 'YY')
    all_lt3_YY.extend(tstamp_lt3)
    all_lt4_YY.extend(tstamp_lt4)
    offsets_YY.extend(np.zeros(np.shape(tstamp_lt3))+ correction_time)
    offsets_ch1_YY.extend(np.zeros(np.shape(tstamp_lt4))+ start_offset_ch1)
    
for d in XX_days_new_timings:

    tstamp_lt3,tstamp_lt4 = Purify_XX.tstamps_for_both_setups(d,contains = 'XX')
    all_lt3_XX.extend(tstamp_lt3)
    all_lt4_XX.extend(tstamp_lt4)
    offsets_XX.extend(np.zeros(np.shape(tstamp_lt3))+ correction_time)
    offsets_ch1_XX.extend(np.zeros(np.shape(tstamp_lt4))+ start_offset_ch1) 

In [ ]:
### load the data
Purify_XX.load_raw_data(all_lt3_XX,all_lt4_XX)
Purify_XX.correct_pq_times(offsets = offsets_XX, offsets_ch1 = offsets_ch1_XX)
Purify_YY.load_raw_data(all_lt3_YY,all_lt4_YY)
Purify_YY.correct_pq_times(offsets = offsets_YY, offsets_ch1 = offsets_ch1_YY)
Purify_ZZ.load_raw_data(all_lt3_ZZ,all_lt4_ZZ)
Purify_ZZ.correct_pq_times(offsets = offsets_ZZ, offsets_ch1 = offsets_ch1_ZZ)

In [ ]:
def sweep_parameter(param_name,sweep_pts,apply_ROC, plot_all = False):
    
    ## get correlation probabilties
    psi_m_XX,psi_p_XX,psi_m_XX_corrs,psi_p_XX_corrs = Purify_XX.sweep_filter_parameter_vs_correlations(param_name,sweep_pts,
                                                                         apply_ROC = apply_ROC,do_plot= False)
    psi_m_YY,psi_p_YY,psi_m_YY_corrs,psi_p_YY_corrs = Purify_YY.sweep_filter_parameter_vs_correlations(param_name,sweep_pts,
                                                                         apply_ROC = apply_ROC,do_plot= False)
    psi_m_ZZ,psi_p_ZZ,psi_m_ZZ_corrs,psi_p_ZZ_corrs = Purify_ZZ.sweep_filter_parameter_vs_correlations(param_name,sweep_pts,
                                                                         apply_ROC = apply_ROC,do_plot= False)       
    ## calculate fidelities
    psi_m_F = (psi_m_XX[1] + psi_m_YY[1] + psi_m_ZZ[1]+1)/4.
    psi_p_F = (psi_p_XX[1] + psi_p_YY[1] + psi_p_ZZ[1]+1)/4.
    
    print psi_m_YY[1]
    print psi_m_ZZ[1]
    
    ## calciulate error bars
    psi_m_F_u = np.sqrt(psi_m_XX[2]**2 + psi_m_YY[2]**2 + psi_m_ZZ[2]**2)/4.
    psi_p_F_u = np.sqrt(psi_p_XX[2]**2 + psi_p_YY[2]**2 + psi_p_ZZ[2]**2)/4.
    
    # calculate ebits    
    ebits_p, ebits_p_u = pa.calculate_ebits(psi_m_YY,psi_m_ZZ,psi_m_XX_corrs)
    ebits_m, ebits_m_u = pa.calculate_ebits(psi_p_YY,psi_p_ZZ,psi_p_XX_corrs)
    
    psi_F = 0.5*(psi_m_F+psi_p_F)
    psi_F_u = 0.5*np.sqrt(psi_m_F_u**2+psi_p_F_u**2)
    
    fig  = plt.figure()
    ax = plt.subplot()
    ax.set_xlabel(param_name)
    ax.set_ylabel('Fidelity')
    plt.errorbar(sweep_pts,psi_m_F,psi_m_F_u,label = '-')
    plt.errorbar(sweep_pts,psi_p_F,psi_p_F_u,label = '+')
    plt.legend()
    plt.show()
    plt.close('all')
 
    fig  = plt.figure()
    ax = plt.subplot()
    ax.set_xlabel(param_name)
    ax.set_ylabel('Fidelity (avg)')
    plt.errorbar(sweep_pts,psi_F,psi_F_u)
    plt.show()
    plt.close('all')
    
    fig  = plt.figure()
    ax = plt.subplot()
    ax.set_xlabel(param_name)
    ax.set_ylabel('ebits')

    plt.errorbar(sweep_pts,ebits_p,ebits_p_u,label = '+')
    plt.errorbar(sweep_pts,ebits_m,ebits_m_u,label = '-')
    plt.legend()
    plt.show()
    plt.close('all')
    
    ### get the rate for any given data point
    ### keep in mind that this includes ALL down time of the experiment. Some might argue that this is an unfair comparison
    ### we therefore also include a diagram that uses the operation time of the sequence only and the number of times the sequence has run
    ### tomography is excluded from this calculated rate
    total_time = 0
    total_time += Purify_XX.get_total_time() + Purify_YY.get_total_time() + Purify_ZZ.get_total_time()
    total_counts = psi_m_XX[0]+psi_p_XX[0]+psi_m_YY[0]+psi_p_YY[0]+psi_m_ZZ[0]+psi_p_ZZ[0]
    fig  = plt.figure()
    ax = plt.subplot()
    ax.set_xlabel(param_name)
    ax.set_ylabel('Rate (Hz)')
    plt.plot(sweep_pts,total_counts/total_time, label = 'incl. overhead')
    
    total_time = 0
    total_time += Purify_XX.estimate_sequence_time() + Purify_YY.estimate_sequence_time() + Purify_ZZ.estimate_sequence_time()
    
    plt.plot(sweep_pts,total_counts/total_time, label = 'excl. overhead')
    plt.legend(loc = 2)
    plt.show()
    plt.close('all')
    
    ## plot individual correlations
    if plot_all:
        fig  = plt.figure()
        ax = plt.subplot()
        ax.set_xlabel(param_name)
        ax.set_ylabel('Expectation value')
        plt.errorbar(sweep_pts,psi_m_XX[1],psi_m_XX[2],label = 'XX')
        plt.errorbar(sweep_pts,psi_m_YY[1],psi_m_YY[2],label = 'YY')
        plt.errorbar(sweep_pts,psi_m_ZZ[1],psi_m_ZZ[2],label = 'ZZ')
        plt.title('Psi_minus correlations')
        plt.legend()
        plt.show()
        plt.close('all')
        
        fig  = plt.figure()
        ax = plt.subplot()
        ax.set_xlabel(param_name)
        ax.set_ylabel('Expectation value')
        plt.errorbar(sweep_pts,psi_p_XX[1],psi_p_XX[2],label = 'XX')
        plt.errorbar(sweep_pts,psi_p_YY[1],psi_p_YY[2],label = 'YY')
        plt.errorbar(sweep_pts,psi_p_ZZ[1],psi_p_ZZ[2],label = 'ZZ')
        plt.title('Psi_plus correlations')
        plt.legend()
        plt.show()
        plt.close('all')

In [ ]:
sweep_parameter('bin_w2',np.arange(50,540,50),apply_ROC = True,plot_all = True)

In [ ]:
sweep_parameter('st_start',np.linspace(-4e3,4e3,10)+2773.5e3,apply_ROC = True)

In [ ]:
sweep_parameter('max_reps_w1',np.arange(100,1000,50),apply_ROC = True)

In [ ]:
sweep_parameter('min_cr_lt4_after',np.arange(1,20,2),apply_ROC = True)
sweep_parameter('min_cr_lt3_after',np.arange(1,20,2),apply_ROC = True)

In [ ]:
sweep_parameter('max_dt',np.linspace(2.5e3,40e3,10),apply_ROC = True)

In [ ]:
sweep_parameter('st_len',np.linspace(3e3,40e3,10),apply_ROC = True)